#import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install plotly_express
!pip install geopandas
!pip install pomegranate

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
from pomegranate import *
from sklearn import preprocessing
import math

In [ ]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_with_wayID'

trips_with_wayId_1_10.csv  trips_with_wayId_1_27.csv  trips_with_wayId_8_15.csv
trips_with_wayId_1_11.csv  trips_with_wayId_1_28.csv  trips_with_wayId_8_16.csv
trips_with_wayId_1_12.csv  trips_with_wayId_1_29.csv  trips_with_wayId_8_17.csv
trips_with_wayId_1_13.csv  trips_with_wayId_1_2.csv   trips_with_wayId_8_18.csv
trips_with_wayId_1_14.csv  trips_with_wayId_1_30.csv  trips_with_wayId_8_19.csv
trips_with_wayId_1_15.csv  trips_with_wayId_1_31.csv  trips_with_wayId_8_20.csv
trips_with_wayId_1_16.csv  trips_with_wayId_1_3.csv   trips_with_wayId_8_21.csv
trips_with_wayId_1_17.csv  trips_with_wayId_1_4.csv   trips_with_wayId_8_22.csv
trips_with_wayId_1_18.csv  trips_with_wayId_1_5.csv   trips_with_wayId_8_23.csv
trips_with_wayId_1_19.csv  trips_with_wayId_1_6.csv   trips_with_wayId_8_24.csv
trips_with_wayId_1_1.csv   trips_with_wayId_1_7.csv   trips_with_wayId_8_25.csv
trips_with_wayId_1_21.csv  trips_with_wayId_1_8.csv   trips_with_wayId_8_26.csv
trips_with_wayId_1_22.csv  trips_with_wa

##get trip data

In [ ]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'
trips_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/way_labeled_decrease_NE_2019'
way_id_on_trips_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips'

In [ ]:
dfs = []
day_len = []
day_trip_count = []
days = range(1,10)
days = range(10,11)
# days = range(1,32)
months = range(1,2)
# months = range(8,9)

for month in months:
  for day in days:
    try:
      sp_seq_df = pd.read_csv(trips_data_path + f'/way_labeled_decrease_2019_{month}_{day}.csv', parse_dates=['time_stamp'])
    except:
      pass

    sp_seq_df = sp_seq_df.sort_values(by=['trip_id', 'time_stamp'])

    trip_count = sp_seq_df.trip_id.value_counts()
    trip_count = trip_count[trip_count >= 12]
    sp_seq_df = sp_seq_df[sp_seq_df.trip_id.isin(trip_count.index)]
    sp_seq_df['month'] = month
    sp_seq_df['day'] = day
    day_len.append(len(sp_seq_df))
    day_trip_count.append(len(trip_count))
    dfs.append(sp_seq_df)
print(day_len)
print(day_trip_count)
df = pd.concat(dfs, ignore_index=True)
df

# df = df[['trip_id', 'way_id', 'day', 'month','route_cluster']]
df.head()

[113042]
[5121]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,way_id,dist,label,month,day
0,2019-01-10 02:27:44,003000900000000011770105654,14.667234,101.434479,41,7.0,0,783299717,0.003591,1126,1,10
1,2019-01-10 02:34:44,003000900000000011770105654,14.692733,101.471751,43,7.0,0,783299714,0.000780,1126,1,10
2,2019-01-10 02:37:44,003000900000000011770105654,14.712998,101.489431,50,7.0,0,783299711,0.002756,1126,1,10
3,2019-01-10 02:44:44,003000900000000011770105654,14.759171,101.517421,49,7.0,0,82374801,0.003864,1149,1,10
4,2019-01-10 02:46:44,003000900000000011770105654,14.771691,101.520754,41,7.0,0,783299693,0.003002,1127,1,10


In [ ]:
df_labeled = df

#func score

In [ ]:
state_names = []
uniq_label = df_labeled.label.unique()
for label in uniq_label:
  state_names.append(str(label))
state_names[:5]

X = []
labels = []
trip_df = df_labeled
for month in months:
  for day in days:
    day_df = trip_df[(df.day == day) & (trip_df.month == month)]
    uniq_trips = day_df.trip_id.unique()
    for trip_id in uniq_trips:
      trip = day_df[day_df.trip_id==trip_id]
      X.append(trip.way_id.to_numpy().astype(str))
      # labels.append(['None-start']+[str(l) for l in trip.route_cluster.to_list()])
      labels.append([str(l) for l in trip.label.to_list()])
print(X[0][:5])
print(labels[0][:5])
print(len(X))
print(len(labels))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



['686957536' '686912257' '686911019' '686910206' '308207967']
['3092', '3092', '3092', '3092', '2059']
2961
2961


In [ ]:
import random
random.seed(42)

X2 = X.copy()
labels2 = labels.copy()

temp = list(zip(X2, labels2))
random.shuffle(temp)
X2, labels2 = zip(*temp)
X2 = list(X2)
labels2 = list(labels2)

print(X[0][:5])
print(X2[0][:5])

print(labels[0][:5])
print(labels2[0][:5])

['686957536' '686912257' '686911019' '686910206' '308207967']
['150574566' '164017910' '163916776' '121529743' '121529742']
['3092', '3092', '3092', '3092', '2059']
['1117', '1110', '1110', '1057', '1057']


In [ ]:
way_seq = df_labeled.way_id.to_list()
not_stop_mask = [True]+[(a != b) for a, b in zip(way_seq[:-1], way_seq[1:])]
df_labeled = df_labeled.loc[not_stop_mask]

way_df = df_labeled[['way_id', 'lat', 'lon']]
way_df = way_df.drop_duplicates(subset='way_id')

sp_list = set(map(str, way_df['way_id'].tolist()))

In [ ]:
total = len(X2)
fold = int(total/10)
start = 0
starts = [] # [0, 952, 1904, 2856, 3808, 4760, 5712, 6664, 7616, 8568]
for i in range(10):
  starts.append(start)
  start+=fold

##for pomegranate

In [ ]:
accuracy_train = []
accuracy_test = []

X_train = X2[:start]+X2[start+fold:]
X_test = X2[start:start+fold]
labels_train = labels2[:start]+labels2[start+fold:]
labels_test = labels2[start:start+fold]

# X_train = X2
# labels_train = labels2
uniq_sp = set()
for seq in X_train:
  for sp in seq:
    uniq_sp.add(sp)
uniq_label = set()

for label in labels_train:
  for uniq in list(set(label)):
    uniq_label.add(uniq)

new_label = []
for label2 in labels_train:
  new_label.append(['None-start']+label2)
labels_train = new_label

unseen_sp = sp_list - uniq_sp
print(len(unseen_sp))
X_train.append(np.array(list(unseen_sp)))
labels_train.append(['-1'] * len(unseen_sp))

model = HiddenMarkovModel.from_samples(
    DiscreteDistribution, 
    n_components=len(uniq_label), 
    X=X_train, 
    labels=labels_train,
    algorithm='labeled',
    state_names=list(uniq_label), 
    inertia=0.001,
    # max_iterations=10,
    max_iterations=1,
    n_jobs=-1
)


0


In [ ]:
# save_model = model.to_json()

In [ ]:
# import json
# with open('/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips/model_1_1.json','w') as f:
#   json.dump(save_model,f)

In [ ]:
# f = open('/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips/model_1_1.json', "r")
# load_model = json.load(f)
# f.close()
# test = HiddenMarkovModel.from_json(load_model)

In [ ]:
# X_test = X2
# labels_test = labels2

In [ ]:
def score(model, seq, label):
  p = []
  prob, path = model.viterbi(seq)
  for s in path[1:]:
    p.append(s[1].name)
  tp = 0
  ###
  # for _state_name, _label in zip(p, label):
  #   if _state_name == _label:
  #     tp+=1
  ###
  # for i, state in enumerate(p):
  #   try:
  #     interval = label[i:i+10]
  #   except:
  #     interval = label[i:]
  #   if state in interval:
  #     tp+=1
  # ### precision
  # total = 0
  # for _state_name, _label in zip(p, label):
  #   if _state_name == '-1':
  #     continue
  #   if _state_name == _label:
  #     tp+=1
  #   total+=1
  ### recal
  # total = 0
  # for _state_name, _label in zip(p, label):
  #   if _state_name == '-1' and _label == '-1':
  #     continue
  #   elif _state_name == _label:
  #     tp+=1
  #   total+=1
  ### precision 10 interval
  total = 0
  for i, state in enumerate(p):
    if state == '-1':
      continue
    try:
      interval = label[i:i+10]
    except:
      interval = label[i:]
    if state in interval:
      tp+=1
    total+=1
  ###
  # total = len(label)
  return tp/total

In [ ]:
def score(model, seq, label):
  p = []
  prob, path = model.viterbi(seq)
  for s in path[1:]:
    p.append(s[1].name)
  
  K = 10
  tp = 0
  total = len(seq)
  hit_list = []
  for i in range(0, len(seq)):
    try:
      points, label_list = p[i:i+K], label[i:i+K]
      count_total = K
    except:
      points, label_list = p[i:], label[i:]
      count_total = len(p[i:])
    count = 0
    for s in points:
      if s in label_list:
        count += 1
    tp += count/count_total

    true_count = 0
    if tp > 0:
      true_count += 1
      hit_list.append(True)
    else:
      true_count += 1
      hit_list.append(False)

    n1=0
    false_count=0
    for i, true in enumerate(hit_list):
      if true==True:
        for not_true in hit_list[i:]:
          if not not_true:
            n1 += 1

    if true_count == 0:
      auc = 0
    elif false_count == 0:
      auc = 1
    else:
      n = true_count*false_count
      auc = n1/n
  return tp/total, auc

In [ ]:
X_test = X
labels_test = labels

In [ ]:
topk_train = []
topk_test = []
hitk_train = []
hitk_test = []
auc_train = []
auc_test = []
fail_count = 0

topk_train = []
hitk_train = []
auc_train = []
for percent in [0.25, 0.5, 0.75, 0.9, 1]:
  top = 0
  hit = 0
  auc = 0
  for seq, label in zip(X_train, labels_train):
    try:
      sc, asc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
    except:
      print("Fail!!", end="\t")
      fail_count = fail_count + 1
      continue
      
    if sc > 0:
      top += sc
      hit += 1
    auc += asc

  train_score = top/len(X_train)
  topk_train.append(train_score)
  train_score = hit/len(X_train)
  hitk_train.append(train_score)
  train_score = auc/len(X_train)
  auc_train.append(train_score)


topk_test = []
hitk_test = []
auc_test = []
for percent in [0.25, 0.5, 0.75, 0.9, 1]:
  top = 0
  hit = 0
  auc = 0
  for seq, label in zip(X_test, labels_test):
    try:
      sc, asc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
    except:
      print("Fail!!", end="\t")
      fail_count = fail_count + 1
      continue
      
    if sc > 0:
      top += sc
      hit += 1
    auc += asc

  test_score = top/len(X_test)
  topk_test.append(test_score)
  test_score = hit/len(X_test)
  hitk_test.append(test_score)
  test_score = auc/len(X_test)
  auc_test.append(test_score)

print("=== auc")
print(f'Train score')
print(f'\t25% trip traverse : {auc_train[0]}')
print(f'\t50% trip traverse : {auc_train[1]}')
print(f'\t75% trip traverse : {auc_train[2]}')
print(f'\t90% trip traverse : {auc_train[3]}')
print(f'\t100%trip traverse : {auc_train[4]}') 
print(f'Test score')
print(f'\t25% trip traverse : {auc_test[0]}')
print(f'\t50% trip traverse : {auc_test[1]}')
print(f'\t75% trip traverse : {auc_test[2]}')
print(f'\t90% trip traverse : {auc_test[3]}')
print(f'\t100%trip traverse : {auc_test[4]}')

print("=== top k")
print(f'Train score')
print(f'\t25% trip traverse : {topk_train[0]}')
print(f'\t50% trip traverse : {topk_train[1]}')
print(f'\t75% trip traverse : {topk_train[2]}')
print(f'\t90% trip traverse : {topk_train[3]}')
print(f'\t100%trip traverse : {topk_train[4]}') 
print(f'Test score')
print(f'\t25% trip traverse : {topk_test[0]}')
print(f'\t50% trip traverse : {topk_test[1]}')
print(f'\t75% trip traverse : {topk_test[2]}')
print(f'\t90% trip traverse : {topk_test[3]}')
print(f'\t100%trip traverse : {topk_test[4]}')

print("=== hit@k")
print(f'Train score')
print(f'\t25% trip traverse : {hitk_train[0]}')
print(f'\t50% trip traverse : {hitk_train[1]}')
print(f'\t75% trip traverse : {hitk_train[2]}')
print(f'\t90% trip traverse : {hitk_train[3]}')
print(f'\t100%trip traverse : {hitk_train[4]}') 
print(f'Test score')
print(f'\t25% trip traverse : {hitk_test[0]}')
print(f'\t50% trip traverse : {hitk_test[1]}')
print(f'\t75% trip traverse : {hitk_test[2]}')
print(f'\t90% trip traverse : {hitk_test[3]}')
print(f'\t100%trip traverse : {hitk_test[4]}')
print('fail seq = ',fail_count)

Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	=== auc
Train score
	25% trip traverse : 0.9999703184826808
	50% trip traverse : 0.9999703184826808
	75% trip traverse : 0.9999703184826808
	90% trip traverse : 0.9999703184826808
	100%trip traverse : 0.9999703184826808
Test score
	25% trip traverse : 1.0
	50% trip traverse : 1.0
	75% trip traverse : 1.0
	90% trip traverse : 1.0
	100%trip traverse : 1.0
=== top k
Train score
	25% trip traverse : 0.3106623339983903
	50% trip traverse : 0.5371119593087453
	75% trip traverse : 0.6747841688330568
	90% trip traverse : 0.7269246194760739
	100%trip traverse : 0.7632171345598957
Test score
	25% trip traverse : 0.4144794815618894
	50% trip traverse : 0.6268417380648675
	75% trip traverse : 0.7286216689025243
	90% trip traverse : 0.7658177264858927
	100%trip traverse : 0.7874623403188631
=== hit@k
Train score
	25% trip traverse : 0.9999703184826808
	50% trip traverse : 0.9999703184826808
	75% trip traverse : 0.9999703184826808
	90% trip traverse : 0.99997031848

In [ ]:
def top_k_score(model, seq, label):
  p = []
  prob, path = model.viterbi(seq)
  for s in path[1:]:
    p.append(s[1].name)
  
  K = 5
  tp = 0
  total = len(seq)
  for i in range(0, len(seq)):
    try:
      points, label_list = p[i:i+K], label[i:i+K]
      count_total = K
    except:
      points, label_list = p[i:], label[i:]
      count_total = len(p[i:])
    count = 0
    for s in points:
      if s in label_list:
        count += 1
    tp += count/count_total
  return tp/total

def auc_score(model, seq, label):
  p = []
  prob, path = model.viterbi(seq)
  for s in path[1:]:
    p.append(s[1].name)

  hit_list = []
  true_count=0
  for pp, l in zip(p, label):
    value = (pp == l)
    if value:
      true_count += 1
    hit_list.append(value)
  
  n1=0
  false_count=0
  for i, true in enumerate(hit_list):
    if true==True:
      for not_true in hit_list[i:]:
        if not not_true:
          n1 += 1
    else:
      false_count += 1

  if true_count == 0:
    return 0
  if false_count == 0:
    return 1
  n = true_count*false_count
  return n1/n

In [ ]:
topk_train = []
topk_test = []
hitk_train = []
hitk_test = []
auc_train = []
auc_test = []
fail_count = 0

topk_train = []
hitk_train = []
auc_train = []
for percent in [0.25, 0.5, 0.75, 0.9, 1]:
  top = 0
  hit = 0
  auc = 0
  for seq, label in zip(X_train, labels_train):
    try:
      sc = top_k_score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
    except:
      print("Fail!!", end="\t")
      fail_count = fail_count + 1
      continue
      
    if sc > 0:
      top += sc
      hit += 1

    try:
      asc = auc_score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
    except:
      print("Fail!!", end="\t")
      fail_count = fail_count + 1
      continue

    auc += asc

  train_score = top/len(X_train)
  topk_train.append(train_score)
  train_score = hit/len(X_train)
  hitk_train.append(train_score)
  train_score = auc/len(X_train)
  auc_train.append(train_score)


topk_test = []
hitk_test = []
auc_test = []
for percent in [0.25, 0.5, 0.75, 0.9, 1]:
  top = 0
  hit = 0
  auc = 0
  for seq, label in zip(X_test, labels_test):
    try:
      sc = top_k_score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
    except:
      print("Fail!!", end="\t")
      fail_count = fail_count + 1
      continue
    
    if sc > 0:
      top += sc
      hit += 1

    try:
      asc = auc_score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
    except:
      print("Fail!!", end="\t")
      fail_count = fail_count + 1
      continue

    auc += asc

  test_score = top/len(X_test)
  topk_test.append(test_score)
  test_score = hit/len(X_test)
  hitk_test.append(test_score)
  test_score = auc/len(X_test)
  auc_test.append(test_score)

print("=== auc")
print(f'Train score')
print(f'\t25% trip traverse : {auc_train[0]}')
print(f'\t50% trip traverse : {auc_train[1]}')
print(f'\t75% trip traverse : {auc_train[2]}')
print(f'\t90% trip traverse : {auc_train[3]}')
print(f'\t100%trip traverse : {auc_train[4]}') 
print(f'Test score')
print(f'\t25% trip traverse : {auc_test[0]}')
print(f'\t50% trip traverse : {auc_test[1]}')
print(f'\t75% trip traverse : {auc_test[2]}')
print(f'\t90% trip traverse : {auc_test[3]}')
print(f'\t100%trip traverse : {auc_test[4]}')

print("=== top k")
print(f'Train score')
print(f'\t25% trip traverse : {topk_train[0]}')
print(f'\t50% trip traverse : {topk_train[1]}')
print(f'\t75% trip traverse : {topk_train[2]}')
print(f'\t90% trip traverse : {topk_train[3]}')
print(f'\t100%trip traverse : {topk_train[4]}') 
print(f'Test score')
print(f'\t25% trip traverse : {topk_test[0]}')
print(f'\t50% trip traverse : {topk_test[1]}')
print(f'\t75% trip traverse : {topk_test[2]}')
print(f'\t90% trip traverse : {topk_test[3]}')
print(f'\t100%trip traverse : {topk_test[4]}')

print("=== hit@k")
print(f'Train score')
print(f'\t25% trip traverse : {hitk_train[0]}')
print(f'\t50% trip traverse : {hitk_train[1]}')
print(f'\t75% trip traverse : {hitk_train[2]}')
print(f'\t90% trip traverse : {hitk_train[3]}')
print(f'\t100%trip traverse : {hitk_train[4]}') 
print(f'Test score')
print(f'\t25% trip traverse : {hitk_test[0]}')
print(f'\t50% trip traverse : {hitk_test[1]}')
print(f'\t75% trip traverse : {hitk_test[2]}')
print(f'\t90% trip traverse : {hitk_test[3]}')
print(f'\t100%trip traverse : {hitk_test[4]}')
print('fail seq = ',fail_count)

Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	=== auc
Train score
	25% trip traverse : 0.9999703184826808
	50% trip traverse : 0.9999703184826808
	75% trip traverse : 0.9999703184826808
	90% trip traverse : 0.9999703184826808
	100%trip traverse : 0.9999703184826808
Test score
	25% trip traverse : 0.48349909902609894
	50% trip traverse : 0.48912250601745993
	75% trip traverse : 0.48340412932793053
	90% trip traverse : 0.48116360585874784
	100%trip traverse : 0.4849564789583504
=== top k
Train score
	25% trip traverse : 0.5637316317120507
	50% trip traverse : 0.7834670851241559
	75% trip traverse : 0.855443140860398
	90% trip traverse : 0.8786166744795809
	100%trip traverse : 0.8947466811835111
Test score
	25% trip traverse : 0.6001912646010711
	50% trip traverse : 0.7568654688694654
	75% trip traverse : 0.8086548562849618
	90% trip traverse : 0.8258250707063101
	100%trip traverse : 0.8320303835168281
=== hit@k
Train score
	25% trip traverse : 0.9999703184826808
	50% trip traverse : 0.9999703184826